Replay: https://python.langchain.com/docs/expression_language/cookbook/memory.

In [1]:
import dotenv
dotenv.load_dotenv('../.env')

True

In [2]:
from langchain.chat_models import AzureChatOpenAI
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

model = AzureChatOpenAI(azure_deployment="gpt-4", openai_api_version="2023-09-01-preview", temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)


In [3]:
memory = ConversationBufferMemory(return_messages=True)

In [4]:
memory.load_memory_variables({})

{'history': []}

In [5]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

In [6]:
inputs = {"input": "hi im bob"}
response = chain.invoke(inputs)
response

AIMessage(content='Hello Bob! How can I assist you today?')

In [7]:
memory.save_context(inputs, {"output": response.content})


In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi im bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')]}

In [9]:
inputs = {"input": "whats my name"}
response = chain.invoke(inputs)
response

AIMessage(content='Your name is Bob.')

In [10]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi im bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')]}